<a href="https://colab.research.google.com/github/abhijith2021/Fetal-health-prediction-from-cardiotocographic-data/blob/main/lstm_fetus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Create CSV database

# In[1]:


import pandas as pd
import wfdb
from os import listdir
from tqdm import tqdm


# In[2]:


def get_all_records():
    rec_list = []
    for file in listdir("/content/drive/MyDrive/ctu-chb-intrapartum-cardiotocography-database-1.0.0"):
        rec = file[:file.find('.')]
        try:
            rec = int(rec)
            rec_list.append(rec)
        except:
            pass
    rec_list = [str(i) for i in rec_list]
    return rec_list

def create_signals_database(rec):
    sample = wfdb.rdsamp("/content/drive/MyDrive/ctu-chb-intrapartum-cardiotocography-database-1.0.0/%s" % rec)
    df = pd.DataFrame(sample[0], columns=['FHR','UC'])
    df.index.name = 'seconds'
    df.to_csv('database/signals/%s.csv' % rec)

def create_ann_dataframe(rec):
    sample = wfdb.rdsamp("/content/drive/MyDrive/ctu-chb-intrapartum-cardiotocography-database-1.0.0/%s" % rec)
    ann = sample[1]['comments'][1:]
    ann_dic = {}
    for a in ann:
        if '--' in a:
            ann.remove(a)

    for a in ann:
        key = a[:a.find('  ')]
        if a.find('  ') == -1:
            key = a[:a.find(' ')]
        inv = a[::-1]
        value = inv[:inv.find(' ')][::-1]
        value = float(value)
        ann_dic[key] = [value]

    df1 = pd.DataFrame.from_dict(ann_dic).T
    df1 = df1.rename(columns={0:rec})
    return df1

def append_ann_dataframes(df,df1):
    rec = df1.columns[0]
    df[rec] = df1[rec]
    return df



# In[3]:


df = pd.DataFrame()
for rec in tqdm(get_all_records()):
    create_signals_database(rec)
    df = append_ann_dataframes(df,create_ann_dataframe(rec))
df.to_csv('database/ann_db.csv')

print('DONE!')


# In[ ]:

 18%|█▊        | 200/1104 [01:10<01:37,  9.30it/s]<ipython-input-38-c6435e0ac309>:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
 18%|█▊        | 201/1104 [01:10<01:37,  9.25it/s]<ipython-input-38-c6435e0ac309>:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
 18%|█▊        | 202/1104 [01:10<01:36,  9.32it/s]<ipython-input-38-c6435e0ac309>:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

DONE!


In [ ]:
import pandas as pd
import os
import wfdb
from tqdm import tqdm


def get_all_records():
    rec_list = []
    for file in listdir("/content/drive/MyDrive/ctu-chb"):
        rec = file[:file.find('.')]
        try:
            rec = int(rec)
            rec_list.append(rec)
        except:
            pass
    rec_list = [str(i) for i in rec_list]
    return rec_list


def create_signals_database(rec):
    sample = wfdb.rdsamp("/content/drive/MyDrive/ctu-chb/%s" % rec)
    df = pd.DataFrame(sample[0], columns=['FHR', 'UC'])
    df['Patient_ID'] = rec
    df.index.name = 'seconds'
    df.to_csv('database/signals/%s.csv' % rec)


def create_ann_dataframe(rec):
    sample = wfdb.rdsamp("/content/drive/MyDrive/ctu-chb/%s" % rec)
    ann = sample[1]['comments'][1:]
    ann_dic = {}
    for a in ann:
        if '--' in a:
            ann.remove(a)

    for a in ann:
        key = a[:a.find('  ')]
        if a.find('  ') == -1:
            key = a[:a.find(' ')]
        inv = a[::-1]
        value = inv[:inv.find(' ')][::-1]
        value = float(value)
        ann_dic[key] = [value]

    df1 = pd.DataFrame.from_dict(ann_dic).T
    df1 = df1.rename(columns={0: rec})
    return df1


def append_ann_dataframes(df, df1):
    rec = df1.columns[0]
    df[rec] = df1[rec]
    return df


def merge_signals_csv():
    all_files = [f for f in listdir('database/signals/') if f.endswith('.csv')]
    df = pd.read_csv('database/signals/' + all_files[0])
    for filename in tqdm(all_files[1:]):
        df_temp = pd.read_csv('database/signals/' + filename)
        df = pd.concat([df, df_temp], ignore_index=True)
    df.to_csv('merged_signals.csv', index=False)


# Create individual signal and annotation DataFrames
for rec in tqdm(get_all_records()):
    create_signals_database(rec)
    create_ann_dataframe(rec)

# Append annotation data to signal DataFrames
df = pd.DataFrame()
for rec in get_all_records():
    df = append_ann_dataframes(df, create_ann_dataframe(rec))
df.to_csv('database/ann_db.csv')

# Merge all signal CSV files
merge_signals_csv()

print('DONE!')


100%|██████████| 600/600 [00:51<00:00, 11.68it/s]
<ipython-input-47-6c4504b69e34>:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
<ipython-input-47-6c4504b69e34>:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rec] = df1[rec]
<ipython-input-47-6c4504b69e34>:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame,

DONE!


In [ ]:
!pip install keras-tuner
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from kerastuner import HyperModel, RandomSearch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

ctg_data = pd.read_csv("/content/merged_signals.csv")

ctg_features = ["seconds", "FHR", "UC"]
target = "FHR"

label_encoder = LabelEncoder()
ctg_data[target] = label_encoder.fit_transform(ctg_data[target])

X_train, X_val, y_train, y_val = train_test_split(
    ctg_data[ctg_features],
    ctg_data[target],
    test_size=0.2,
    random_state=42,
)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

X_train_scaled = np.expand_dims(X_train_scaled, axis=2)
X_val_scaled = np.expand_dims(X_val_scaled, axis=2)

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                  return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)))
    model.add(LSTM(units=hp.Int('second_layer_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']), metrics=['accuracy'])
    return model

tuner = RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1,
    executions_per_trial=1,
    directory="/content/hyperparameter_tuning_results",
    project_name="fetal_heartbeat_predictor",
    **{"max_consecutive_failed_trials": 5}
)

tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_val_scaled, y_val), verbose=2)

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)))
model.add(LSTM(32))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_val_scaled, y_val))

y_pred_prob = model.predict(X_val_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='weighted')
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Trial 1 Complete [01h 01m 30s]
val_accuracy: 0.6838852763175964

Best val_accuracy So Far: 0.6838852763175964
Total elapsed time: 01h 01m 30s
Epoch 1/20
26215/26215 [==============================] - 209s 8ms/step - loss: 2.9875 - accuracy: 0.3149 - val_loss: 2.2773 - val_accuracy: 0.4042
Epoch 2/20
26215/26215 [==============================] - 204s 8ms/step - loss: 2.0291 - accuracy: 0.4497 - val_loss: 1.8380 - val_accuracy: 0.4860
Epoch 3/20
26215/26215 [==============================] - 201s 8ms/step - loss: 1.7210 - accuracy: 0.5105 - val_loss: 1.6081 - val_accuracy: 0.5488
Epoch 4/20
26215/26215 [==============================] - 207s 8ms/step - loss: 1.5631 - accuracy: 0.5465 - val_loss: 1.5415 - val_accuracy: 0.5153
Epoch 5/20
26215/26215 [==============================] - 205s 8ms/step - loss: 1.4547 - accuracy: 0.5745 - val_loss: 1.4651 - val_accuracy: 0.5315
Epoch 6/20
26215/26215 [==============================] - 201s 8ms/step - loss: 1.3677 - accuracy: 0.5945 - val_loss: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7110
F1 Score: 0.6257
Precision: 0.5955
Recall: 0.7110
